In [1]:
%%capture
!pip install -r requirements.txt --user

In [2]:
path_to_MyModule = '..'

import sys
sys.path.insert(0, path_to_MyModule) 

import pandas as pd

from MyModule.GeneralFunctions import *
from MyModule.SummarizationFunctions import *
from MyModule.SamplingFunctions import *

In [3]:
df = pd.read_excel('..\datos.xlsx')[['ID','texto','desafio']]

In [4]:
# Limpieza
df.drop_duplicates(subset='texto', inplace=True)

# Quitando texto de mas en columna "desafio"
df['desafio'] = df['desafio'].apply(lambda x: re.findall('[0-9]+', x)[0])

# A str
df['texto'] = df['texto'].astype(str)

# Summary of each topic

Abstractive summary with T5

In [11]:
t5 = T5Summary()

desafios = df['desafio'].unique()
for des in desafios:
    text = '\n'.join(df[df['desafio']==des]['texto'])
    print(f'Desafio {des}:\n{t5.generate_summary(text, num_beams=10)}\n\n')

ckpt:
josmunpen/mt5-small-spanish-summarization
Desafio 15:
['el estado quiere ayudar a garantizar el derecho a la alimentación y la salud', 'el estado debe ayudar a garantizar el derecho a la alimentación y la salud']


Desafio 13:
['partidos de izquierda "me siento mas identificado con los partidos de izquierda"', 'partidos de izquierda luchan por una distribución de la riqueza más justa']


Desafio 14:
['a ninguna persona, ninguna figura, ninguna figura que me genera esas emociones', 'ella de carloto "no tengo referente de la política argentina"']


Desafio 12:
['la izquierda moderada', 'la izquierda moderada, una persona de izquierda moderada']


Desafio 16:
['los subsidios a las empresas de transportes no son las mismas que cuando fundaron', 'los subsidios a las empresas de transportes no son las mismas que cuando fundaron']


Desafio 17:
['la hora de cursada sea menor', 'la hora de cursada sea menor']


Desafio 18:
['los horarios de cursada', 'los horarios de cursada']




# Select most representative documents of each topic

In [4]:
from MyModule.SummarizationFunctions import MostRepresentativeDocs

pp_object = Preprocess(lemma=False)
mrd = MostRepresentativeDocs()

In [5]:
all_res = {}
for des in df['desafio'].unique():
    data = df[df['desafio']==des]['texto'].values
    result = mrd.get_representatives(data, pp_object, 3)
    all_res[des] = result

In [6]:
all_res['17'][0]

[('mas opciones de comida vegetarias tanto en bandeja como en el kiosco',
  0.8862039),
 ('Mas variedad de comida vegetariana', 0.82250243),
 ('Más menús de opciones vegetarianas y veganas. ', 0.81061655),
 ('más comidas hechas como tartas, ensaladas, etc', 0.68575925),
 ('Comida sin harina en el kiosco ', 0.64939505)]

# Sample representative docs

In [7]:
desafio = '12'

documents = df[df['desafio']==desafio]['texto'].values

pp_object = Preprocess(lemma=False, stopwords=False, alphanumeric=False, join=False)

sr = SampleRepresentatives()
res = sr.get_sample(documents, pp_object, n_clusters=5)

In [8]:
print(f'Desafio: {desafio}')
for key, value in res.items():
    print(f'\nCluster: {key}\n')
    for v in value[:3]:
        print(v)

Desafio: 12

Cluster: 1

No tengo una posición muy formada, pero me identifico con el liberalismo. Completamente liberal en lo social, y relativamente en lo económico. 
No me siento identificada con ningún partido político.En materia económica simpatizo con las ideas liberales.
No puedo definirme ideológicamente en estos términos porque desconozco a qué hace referencia 

Cluster: 2

Nacionalista y popular. Porque considero que la economía se debe manejar en función de los intereses de las mayorías, las clases populares y los intereses nacionales, en defensa de la patria y sus recursos, y defender un mercado interno fuerte con buenos salarios 
Creo que las narrativas en términos politicos y económicos actuales no están a la altura del mundo actual. Son ideologías viejas surgidas en contextos totalmente diferentes, no responden a las necesidades del mundo actual, sus problemáticas y sus avances. Estamos viviendo el inicio de la cuarta revolución industrial y tenemos herramientas sociales